In [13]:
import pandas as pd

final_csv_path = '../../data/refined_dataset.csv'

# Read dataset
df = pd.read_csv(final_csv_path)
df['publish_date'] = df['publish_date'].apply(lambda x: x[:-7])  # keeping only date
df = df[['publish_date', 'content']]
df['id'] = df.index

df.head()

,publish_date,content,id
0,October 20th 2019,به گزارش خبرگزاری شبستان : به نقل از آژانس خ...,0
1,October 20th 2019,کاهش تنوع مدارس، شعار سال‌های اخیر سکانداران آ...,1
2,October 20th 2019,- اخبار ورزشی - صادق ورمزیار در گفت‌وگو ب...,2
3,October 20th 2019,بنا بر اعلام دفتر پلوسی، رئیس مجلس نمایندگان آ...,3
4,October 20th 2019,به گزارش ایمنا و به نقل از فدراسیون جودو، ای...,4


In [12]:
def get_same_days(doc_id):
    return df[df['publish_date'] == df['publish_date'][doc_id]]


get_same_days(666)

,publish_date,content
167,October 18th 2019,نمایش ستونی تصاویر ...
168,October 18th 2019,به گزارش خبرنگار گروه استان های باشگاه خبرن...
169,October 18th 2019,پیرمرد عراقی با حسرت از آن روزها یاد می کند و ...
170,October 18th 2019,به گزارش خبرگزاری برنا از لرستان ،همزمان با او...
171,October 18th 2019,- اخبار استانها - به گزارش خبرگزاری تسنی...
...,...,...
7185,October 18th 2019,به گزارش خبرگزاری صدا و سیما مرکز آبادان؛ حجت ...
7186,October 18th 2019,عمل جراحی بازسازی گونه و صورت بیمار ۳۵ ساله عر...
7187,October 18th 2019,به گزارش سرویس بین الملل خبرگزاری صدا و سیما ...
7188,October 18th 2019,به گزارش خبرگزاری صدا و سیما مرکز مهاباد، ام...


In [ ]:
from __future__ import division
import random
import binascii


num_hashes = 10
max_shingle_id = 2 ** 32 - 1
next_prime = 4294967311


def pick_random_coeffs(k):
    # Create a list of 'k' random values.
    rand_list = []
    while k > 0:
        # Get a random shingle ID.
        rand_index = random.randint(0, max_shingle_id)
        while rand_index in rand_list:  # ensure that each random number is unique
            rand_index = random.randint(0, max_shingle_id)
        rand_list.append(rand_index)
        k -= 1
    return rand_list


doc_shingle_sets = []
total_shingles = 0

for doc in df['content']:
    shingles_in_doc = set()
    words = doc.split(" ")
    for index in range(len(words) - 2):
        shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]
        # Hash the shingle to a 32-bit integer.
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
        shingles_in_doc.add(crc)
    # store shingle set
    doc_shingle_sets.append(shingles_in_doc)
    total_shingles += len(words) - 2

num_docs = len(df.index)


coeff_a = pick_random_coeffs(num_hashes)
coeff_b = pick_random_coeffs(num_hashes)

signatures = []

for i in range(num_docs):
    shingle_set = doc_shingle_sets[i]
    signature = []

    for i in range(num_hashes):
        min_hash_code = next_prime + 1
        for shingle in shingle_set:
            hash_code = (coeff_a[i] * shingle + coeff_b[i]) % next_prime
            if hash_code < min_hash_code:
                min_hash_code = hash_code

        signature.append(min_hash_code)

    signatures.append(signature)

In [ ]:
jac_threshold = 0.5

similar_docs = []

for i in range(num_docs):
    similar_to_i = []

    signature1 = signatures[i]
    for _, row in get_same_days(i).iterrows():
        j = row['id']
        signature2 = signatures[j]
        count = 0
        for k in range(num_hashes):
            if signature1[k] == signature2[k]:
                count += 1
        if (count / num_hashes) > jac_threshold:
            similar_to_i.append(j)

    similar_docs.append(similar_to_i)

In [5]:
duplicates_path = '../../data/duplicates.pickle'
with open(duplicates_path, 'wb') as dups_file:
    pickle.dump(similar_docs, dups_file)